Most existing approaches to visual odometry are based on the following stages.

1. Acquire input images
2. Image correction: apply image processing techniques for lens distortion removal, etc.
3. Feature detection: define interest operators, and match features across frames and construct optical flow field.
        3.1. Use correlation to establish correspondence of two images.
        3.2. Feature extraction and correlation
        3.3. Construct optical flow field (Lucas–Kanade method).
4. Check flow field vectors for potential tracking errors and remove outliers.[7]
5. Estimation of the camera motion from the optical flow.[8][9][10][11]
        Choice 1: Kalman filter for state estimate distribution maintenance.
        Choice 2: find the geometric and 3D properties of the features that minimize a cost function based on the re-projection error between two adjacent images. This can be done by mathematical minimization or random sampling.
6. Periodic repopulation of trackpoints to maintain coverage across the image.

In [10]:
################################Testing################################

In [3]:
%run pinhole_camera.py

## defining directories
poses_dir = 'dataset/poses/00.txt' #for ground truth
img_dir = 'dataset/sequences/00/image_0/'

cam = PinholeCamera(1241.0, 376.0, 718.8560, 718.8560, 607.1928, 185.2157)
vo = VisualOdometry(cam, poses_dir)

traj = np.zeros((600,600,3), dtype=np.uint8)

#drawing trajectories for each image
for img_id in xrange(10):
    img = cv2.imread(img_dir+str(img_id).zfill(6)+'.png', 0)
    
    vo.update(img, img_id)

    cur_t = vo.cur_t
    if(img_id > 2): x, y, z = cur_t[0], cur_t[1], cur_t[2]
    else: x, y, z = 0., 0., 0.
        
    #offset so the 2 trajectories do not overlap
    x_offset = 0
    y_offset = 0
    draw_x, draw_y = int(x)+(290-x_offset), int(z)+(90-y_offset)
    true_x, true_y = int(vo.trueX)+290, int(vo.trueZ)+90

    #predicted trajectory
    cv2.circle(traj, (draw_x,draw_y), 1, (img_id*255/4540,255-img_id*255/4540,0), 1)
    #actual trajectory
    cv2.circle(traj, (true_x,true_y), 1, (0,0,255), 1)
    cv2.rectangle(traj, (10, 20), (600, 60), (0,0,0), -1)
    text = "Coordinates: x=%2fm y=%2fm z=%2fm"%(x,y,z)
    cv2.putText(traj, text, (20,40), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)

#     cv2.imshow('Road facing camera', img)
#     cv2.imshow('Trajectory', traj)
    cv2.waitKey(1)


cv2.imwrite('map.png', traj)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1